In [3]:
# pip install pygam

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 8.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 12.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.12.0
    Uninstalling scipy-1.12.0:
      Successfully uninstalled scipy-1.12.0

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from pygam import GAM, s

In [3]:
import os
import pandas as pd
import torch
import numpy as np
import re
from torch.utils.data import Dataset

class TubingenDatasetHighDim(Dataset):
    def __init__(self, folder_path, meta_path):
        self.data = []
        self.weights = []
        
        # 메타데이터 읽기
        meta_data = pd.read_csv(meta_path, header=None, sep=r'[ \t]+', engine='python')
        meta_dict = {}
        for row in meta_data.itertuples():
            pair_num = f"{int(row[1]):04d}"
            meta_dict[pair_num] = {
                'x_start': int(row[2]),
                'x_end': int(row[3]),
                'y_start': int(row[4]),
                'y_end': int(row[5]),
                'weight': float(row[6])
            }
            
        def read_data_file(file_path):
            try:
                with open(file_path, 'r') as f:
                    first_line = f.readline().strip()

                # CSV 형식 확인
                if ',' in first_line:
                    # 모든 행(row) 읽기
                    df = pd.read_csv(file_path, header=None)  # 헤더가 없는 경우 header=None
                    return df.values.astype(np.float32)  # NumPy 배열로 변환
                else:
                    # 공백/탭 구분자로 시도
                    data = []
                    with open(file_path, 'r') as f:
                        for line in f:
                            # 공백으로 분리하고 모든 값 사용
                            values = [float(val) for val in line.strip().split()]
                            if values:
                                data.append(values)

                    data_array = np.array(data, dtype=np.float32)
                    return data_array
            except Exception as e:
                print(f"Error reading file {file_path}: {e}")
                return None

        def find_direction_in_file(file_content):
            """파일 전체 내용에서 방향성 찾기"""
            # 텍스트 전처리
            content = file_content.lower()
            
            # x->y 패턴 찾기 (다양한 형식 포함)
            x_to_y_patterns = [
                r'x\s*-+\s*>\s*y',         # x->y, x --> y, x  >  y
                r'x\s*-\s*-\s*>\s*y',      # x - - > y
                r'x\s*-+\s*-+\s*>\s*y',    # x -- > y, x --- > y
                r'x\s*→\s*y',              # x→y (유니코드 화살표)
                r'x\s+to\s+y',             # x to y
                r'y\s*<-\s*x',             # y <- x
                r'y\s*←\s*x',              # y ← x (유니코드 화살표)
                r'ground\s*truth.*x\s*-*>\s*y',  # ground truth x->y
                r'ground\s*truth.*x\s+to\s+y',   # ground truth x to y
                r'ground\s*truth.*x\s*-\s*-\s*>\s*y'  # ground truth x - - > y
            ]
            
            # y->x 패턴 찾기
            y_to_x_patterns = [
                r'y\s*-+\s*>\s*x',         # y->x, y --> x, y  >  x
                r'y\s*-\s*-\s*>\s*x',      # y - - > x
                r'y\s*-+\s*-+\s*>\s*x',    # y -- > x, y --- > x
                r'y\s*→\s*x',              # y→x (유니코드 화살표)
                r'y\s+to\s+x',             # y to x
                r'x\s*<-\s*y',             # x <- y
                r'x\s*←\s*y',              # x ← y (유니코드 화살표)
                r'ground\s*truth.*y\s*-*>\s*x',  # ground truth y->x
                r'ground\s*truth.*y\s+to\s+x',   # ground truth y to x
                r'ground\s*truth.*y\s*-\s*-\s*>\s*x',  # ground truth y - - > x
                r'ground\s*truth.*x\s*<-\s*y'    # ground truth x <- y
            ]
            
            # x->y 패턴 매칭 (대문자 버전 포함)
            for pattern in x_to_y_patterns:
                if re.search(pattern, content) or re.search(pattern.replace('x', 'X').replace('y', 'Y'), file_content):
                    return 1
                    
            # y->x 패턴 매칭 (대문자 버전 포함)
            for pattern in y_to_x_patterns:
                if re.search(pattern, content) or re.search(pattern.replace('x', 'X').replace('y', 'Y'), file_content):
                    return 0
            
            # 매칭되지 않는 경우
            print(f"Warning: Unable to determine direction from text: '{file_content.strip()}'")
            return -1
        
        # 파일 처리
        successful_loads = 0
        skipped_files = []  # 레이블을 결정할 수 없는 파일들 추적
        
        for file_name in sorted(os.listdir(folder_path)):
            if file_name.endswith('.csv') and not file_name.endswith('_des.csv'):
                pair_num = file_name[4:8]
                csv_path = os.path.join(folder_path, file_name)
                des_file_name = file_name.replace('.csv', '_des.csv')
                des_path = os.path.join(folder_path, des_file_name)
                
                if pair_num in meta_dict:
                    meta = meta_dict[pair_num]
                    
                    # 데이터 읽기
                    raw_data = read_data_file(csv_path)
                    
                    if raw_data is not None and len(raw_data) > 0:
                        try:
                            # 메타데이터의 범위에 따라 X와 Y 데이터 분리
                            x_start = meta['x_start'] - 1  # 1-based to 0-based
                            x_end = meta['x_end']
                            y_start = meta['y_start'] - 1  # 1-based to 0-based
                            y_end = meta['y_end']

                            # x_data는 첫 번째 열만 사용
                            x_data = raw_data[:, x_start:x_end]  # 행(row)을 기준으로 범위를 지정하고, 열(column) 명시
                            # y_data는 두 번째 열만 사용
                            y_data = raw_data[:, y_start:y_end]  # 행(row)을 기준으로 범위를 지정하고, 열(column) 명시

                            # 빈 데이터 확인
                            if len(x_data) == 0 or len(y_data) == 0:
                                print(f"Skipping {file_name} due to empty data")
                                continue
                            
                            # 텐서로 변환
                            x_tensor = torch.tensor(x_data, dtype=torch.float32)
                            y_tensor = torch.tensor(y_data, dtype=torch.float32)
                            
                            # des 파일에서 방향성 정보 읽기
                            if os.path.exists(des_path):
                                with open(des_path, 'r') as des_file:
                                    des_content = des_file.read()
                                    label = find_direction_in_file(des_content)
                                    
                                    if label == -1:
                                        skipped_files.append((file_name, des_content.strip()))
                            else:
                                label = -1
                                print(f"\nWarning: No description file found for {file_name}")
                                print("Default label assigned: -1 (unknown)")
                                skipped_files.append((file_name, "No description file"))
                            
                            # 데이터 저장 (레이블이 유효한 경우만)
                            if label != -1:
                                self.data.append({
                                    'x': x_tensor,
                                    'y': y_tensor,
                                    'label': label,
                                    'weight': meta['weight'],
                                    'file_name': file_name
                                })
                                successful_loads += 1
                            
                        except Exception as e:
                            print(f"Data processing error in {file_name}: {e}")
                            continue
        
        print(f"\nSuccessfully loaded {successful_loads} files")
        
        # 레이블 통계 출력
        label_counts = {1: 0, 0: 0, -1: 0}
        for item in self.data:
            label_counts[item['label']] += 1
            '''
        print("\nLabel Distribution:")
        print(f"Label 1 (x→y): {label_counts[1]} samples")
        print(f"Label 0 (y→x): {label_counts[0]} samples")'''
        
        # 스킵된 파일 정보 출력
        if skipped_files:
            print("\nSkipped files due to unclear direction:")
            for file_name, content in skipped_files:
                print(f"\n{file_name}:")
                print(f"Content: {content}")

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        return sample['x'], sample['y'], sample['label'], sample['weight']
    
    def get_dimensions(self):
        """각 데이터셋의 차원 정보를 반환"""
        dimensions = []
        for sample in self.data:
            dimensions.append({
                'file': sample.get('file_name', 'unknown'),
                'x_dim': tuple(sample['x'].shape),
                'y_dim': tuple(sample['y'].shape)
            })
        return dimensions
    
    def get_label_summary(self):
        """데이터셋의 라벨 분포를 반환"""
        label_count = {0: 0, 1: 0, -1: 0}
        file_labels = []
        
        for sample in self.data:
            label = sample['label']
            label_count[label] += 1
            file_labels.append({
                'file': sample['file_name'],
                'label': label,
                'weight': sample['weight']
            })
            
        return {
            'summary': label_count,
            'details': file_labels
        }
    def get_valid_data(self):
        special_cases = ["pair0052.csv", "pair0053.csv", "pair0054.csv", "pair0055.csv", "pair0071.csv", "pair0105.csv"]
        return [
            sample for sample in self.data if sample['file_name'] not in special_cases
        ]
    def sample_out(self):
        for sample in self.data:
            print(f"File: {sample['file_name']}, x shape: {sample['x'].shape}, y shape: {sample['y'].shape}")   

In [4]:
# Dataset 로드
dataset = TubingenDatasetHighDim(
    folder_path='pairs(csv)',
    meta_path='pairs(csv)/pairmeta.csv'
)
valid_data = dataset.get_valid_data()

 Switzerland
""
""
""
 Switzerland)
""
""
""'
 Switzerland
""
""
""
 Switzerland)
""
""
""'
 Switzerland
""
""
""
 Switzerland)
""
""
""'
 corresponding to cows who stopped producing milk before the end of the experiment.
""
""
""
""
""
""
""
""
""
""
""'
Second column: monthly rent in EUR'
" Canada over the period July 1 1972 to December 31 2009. Measured at top of Whistler Gondola: Latitude: 50°04'04.000"" N Longitude: 122°56'50.000"" W Elevation: 1835.00 m "
""
""
""
""
""'
y = Relative Spinal bone mineral density
Source:
bone data set from http://cran.r-project.org/web/packages/ElemStatLearn/ElemStatLearn.pdf
We obtained and preprocessed the data using the following R commands:
install.packages('ElemStatLearn')
library(ElemStatLearn)
for( i in 2:dim(bone)[1] ) { 
  } 
}
remove.packages('ElemStatLearn')
"selected only the columns ""age"" and ""spnbmd"" (Relative Change in Spinal BMD)."
The ground truth should be that age causes relative change in spinal BMD.'

Successfully loaded 10

In [5]:
dim = dataset.get_dimensions()

In [6]:
dim

[{'file': 'pair0001.csv', 'x_dim': (349, 1), 'y_dim': (349, 1)},
 {'file': 'pair0002.csv', 'x_dim': (349, 1), 'y_dim': (349, 1)},
 {'file': 'pair0003.csv', 'x_dim': (349, 1), 'y_dim': (349, 1)},
 {'file': 'pair0004.csv', 'x_dim': (349, 1), 'y_dim': (349, 1)},
 {'file': 'pair0005.csv', 'x_dim': (4177, 1), 'y_dim': (4177, 1)},
 {'file': 'pair0006.csv', 'x_dim': (4177, 1), 'y_dim': (4177, 1)},
 {'file': 'pair0007.csv', 'x_dim': (4177, 1), 'y_dim': (4177, 1)},
 {'file': 'pair0008.csv', 'x_dim': (4177, 1), 'y_dim': (4177, 1)},
 {'file': 'pair0009.csv', 'x_dim': (4177, 1), 'y_dim': (4177, 1)},
 {'file': 'pair0010.csv', 'x_dim': (4177, 1), 'y_dim': (4177, 1)},
 {'file': 'pair0011.csv', 'x_dim': (4177, 1), 'y_dim': (4177, 1)},
 {'file': 'pair0012.csv', 'x_dim': (5000, 1), 'y_dim': (5000, 1)},
 {'file': 'pair0013.csv', 'x_dim': (392, 1), 'y_dim': (392, 1)},
 {'file': 'pair0014.csv', 'x_dim': (392, 1), 'y_dim': (392, 1)},
 {'file': 'pair0015.csv', 'x_dim': (392, 1), 'y_dim': (392, 1)},
 {'file':

In [7]:
def organize_domains(valid_data):
    domains = {}
    for sample in valid_data:
        file_name = sample['file_name']
        if file_name not in domains:
            domains[file_name] = {'x': [], 'y': [], 'label': [], 'weight': []}
        domains[file_name]['x'].append(sample['x'])
        domains[file_name]['y'].append(sample['y'])
        domains[file_name]['label'].append(sample['label'])
        domains[file_name]['weight'].append(sample['weight'])
    return domains

In [8]:
domains = organize_domains(valid_data)

In [18]:
import numpy

def centering(M):
    """
    Calculate the centering matrix
    """
    n = M.shape[0]
    unit = numpy.ones([n, n])
    identity = numpy.eye(n)
    H = identity - unit/n

    return numpy.matmul(M, H)

def gaussian_grammat(x, sigma=None):
    """
    Calculate the Gram matrix of x using a Gaussian kernel.
    If the bandwidth sigma is None, it is estimated using the median heuristic:
    ||x_i - x_j||**2 = 2 sigma**2
    """
    try:
        x.shape[1]
    except IndexError:
        x = x.reshape(x.shape[0], 1)

    xxT = numpy.matmul(x, x.T)
    xnorm = numpy.diag(xxT) - xxT + (numpy.diag(xxT) - xxT).T
    if sigma is None:
        mdist = numpy.median(xnorm[xnorm!= 0])
        sigma = numpy.sqrt(mdist*0.5)


   # --- If bandwidth is 0, add machine epsilon to it
    if sigma==0:
        eps = 7./3 - 4./3 - 1
        sigma += eps

    KX = - 0.5 * xnorm / sigma / sigma
    numpy.exp(KX, KX)
    return KX

def dHSIC_calc(K_list):
    """
    Calculate the HSIC estimator in the general case d > 2, as in
    [2] Definition 2.6
    """
    if not isinstance(K_list, list):
        K_list = list(K_list)

    n_k = len(K_list)

    length = K_list[0].shape[0]
    term1 = 1.0
    term2 = 1.0
    term3 = 2.0/length

    for j in range(0, n_k):
        K_j = K_list[j]
        term1 = numpy.multiply(term1, K_j)
        term2 = 1.0/length/length*term2*numpy.sum(K_j)
        term3 = 1.0/length*term3*K_j.sum(axis=0)

    term1 = numpy.sum(term1)
    term3 = numpy.sum(term3)
    dHSIC = (1.0/length)**2*term1+term2-term3
    return dHSIC

def HSIC(x, y):
    """
    Calculate the HSIC estimator for d=2, as in [1] eq (9)
    """
    n = x.shape[0]
    return numpy.trace(numpy.matmul(centering(gaussian_grammat(x)),centering(gaussian_grammat(y))))/n/n

def dHSIC(*argv):
    assert len(argv) > 1, "dHSIC requires at least two arguments"

    if len(argv) == 2:
        x, y = argv
        return HSIC(x, y)

    K_list = [gaussian_grammat(_arg) for _arg in argv]

    return dHSIC_calc(K_list)

In [19]:
import numpy as np
from pygam import LinearGAM


class GAM:
    
    def __init__(self, n_splines):
        self.n_splines = n_splines
        
    def fit(self, x, y):
        # Check `x` dimensionality
        x = np.array(x)
        assert len(x.shape) == 2, f'`x` should be 2D array. Received {len(x.shape)} dimensional array.'
        
        # Fit the model
        self.model = LinearGAM(n_splines=self.n_splines).gridsearch(x, y) 
        
    def predict(self, x):
        return self.model.predict(x)

In [23]:
def train_and_evaluate_domain(domain, data):
    x = data['x'][0].squeeze().numpy()
    y = data['y'][0].squeeze().numpy()

    # Define params
    n_splines = 200
    
    # Instantiate the models 
    model_xy = GAM(n_splines=n_splines)
    model_yx = GAM(n_splines=n_splines)
    
    # Fit the models
    model_xy.fit(x.reshape(-1, 1), y)
    model_yx.fit(y.reshape(-1, 1), x)
    
    # Generate predictions
    y_pred = model_xy.predict(x.reshape(-1, 1))
    x_pred = model_yx.predict(y.reshape(-1, 1))

    residuals_xy = y - y_pred
    residuals_yx = x - x_pred

    print('Causal direction (HSIC):')
    if HSIC(x, y - y_pred) < HSIC(y, x - x_pred):
        print('X -> Y')
    else:
        print('Y -> X')

In [ ]:
for domain, data in domains.items():
    train_and_evaluate_domain(domain, data)

100% (11 of 11) |########################| Elapsed Time: 0:00:01 Time:  0:00:010:00
100% (11 of 11) |########################| Elapsed Time: 0:00:01 Time:  0:00:010:00
 18% (2 of 11) |####                     | Elapsed Time: 0:00:00 ETA:   0:00:00

Causal direction (HSIC):
X -> Y


100% (11 of 11) |########################| Elapsed Time: 0:00:01 Time:  0:00:010:00
100% (11 of 11) |########################| Elapsed Time: 0:00:01 Time:  0:00:010:00
  9% (1 of 11) |##                       | Elapsed Time: 0:00:00 ETA:   0:00:00

Causal direction (HSIC):
Y -> X


100% (11 of 11) |########################| Elapsed Time: 0:00:01 Time:  0:00:010:00
100% (11 of 11) |########################| Elapsed Time: 0:00:01 Time:  0:00:010:00
  9% (1 of 11) |##                       | Elapsed Time: 0:00:00 ETA:   0:00:01

...

Causal direction (HSIC):
X -> Y


100% (11 of 11) |########################| Elapsed Time: 0:00:02 Time:  0:00:020:00
100% (11 of 11) |########################| Elapsed Time: 0:00:02 Time:  0:00:020:00
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Causal direction (HSIC):
X -> Y


100% (11 of 11) |########################| Elapsed Time: 0:00:06 Time:  0:00:060:00
100% (11 of 11) |########################| Elapsed Time: 0:00:06 Time:  0:00:060:00


Causal direction (HSIC):
